In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
list_of_filenames = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if "png" in filename:
#         print(os.path.join(dirname, filename))
            list_of_filenames.append(f"{os.path.join(dirname, filename)}")
#         print(_)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
for filename in list_of_filenames:
    if 'png' not in filename:
        print(filename)
        list_of_filenames.remove(filename)

In [ ]:
for filename in list_of_filenames:
    if 'png' not in filename:
        print(filename)

In [ ]:
import random
random.shuffle(list_of_filenames)

In [ ]:
list_of_filenames[:20]

In [ ]:
# malignant normal
# for i in list_of_filenames:
list_filenames_unmasked = []
list_filenames_masked = []
for i in list_of_filenames[:10]:
     if "mask" in i:
            print(i)

In [ ]:
# Import necessary tools
import tensorflow as tf
import tensorflow_hub as hub
print("TF Version:", tf.__version__)
print("TF Hub Version:", hub.__version__)

# Check for GPU availability
print("GPU", "available (YESSSSSS!!)" if tf.config.list_physical_devices("GPU") else "not available :(" )

In [ ]:
tf.config.list_physical_devices()

In [ ]:
from IPython.display import Image
Image("/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/malignant (8)_mask.png")

In [ ]:
from IPython.display import Image
Image("/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/normal/normal (130)_mask.png")

In [ ]:
file_dict = {}

for file_path in list_of_filenames:
    # determine the label based on the presence of the category in the file path
    if 'benign' in file_path:
        label = 'benign'
        file_dict[file_path] = label
    elif 'malignant' in file_path:
        label = 'malignant'
        file_dict[file_path] = label
    elif 'normal' in file_path:
        label = 'normal'
        file_dict[file_path] = label
    else:
        label = 'unknown'  # or raise an error, depending on your requirements

In [ ]:
labels_csv = pd.DataFrame(list(file_dict.items()), columns=['file_path', 'label'])
labels_csv.head()

In [ ]:
labels_csv["label"].value_counts().plot.bar(figsize=(20, 10))

In [ ]:
import numpy as np
labels = labels_csv["label"].to_numpy() 
# labels = np.array(labels) # does same thing as above
labels

In [ ]:
# See if number of labels matches the number of filenames
if len(labels) == len(list_of_filenames):
  print("Number of labels matches number of filenames!")
else:
  print("Number of labels does not match number of filenames, check data directories!")

In [ ]:
# Find the unique label values
unique_labels = np.unique(labels)
len(unique_labels)

In [ ]:
# Turn a single label into an array of booleans
print(labels[0])
labels[0] == unique_labels

In [ ]:
# Turn every label into a boolean array
boolean_labels = [label == unique_labels for label in labels]
boolean_labels[:2]

In [ ]:
# Example: Turning boolean array into integers
print(labels[0]) # original label
print(np.where(unique_labels == labels[0])) # index where label occurs
print(boolean_labels[0].argmax()) # index where label occurs in boolean array
print(boolean_labels[0].astype(int)) # there will be a 1 where the sample label occurs

In [ ]:
print(labels[2])
print(boolean_labels[2].astype(int))

In [ ]:
# Setup X & y variables
X = list_of_filenames
y = boolean_labels

In [ ]:
# Set number of images to use for experimenting
NUM_IMAGES = 1578

In [ ]:
# Let's split our data into train and validation sets
from sklearn.model_selection import train_test_split

# Split them into training and validation of total size NUM_IMAGES
X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES],
                                                  y[:NUM_IMAGES],
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

In [ ]:
len(labels_csv)

In [ ]:
# Let's have a geez at the training data
X_train[:2], y_train[:2]

In [ ]:
# Convert image to NumPy array
from matplotlib.pyplot import imread
image = imread(list_of_filenames[42])
image.shape

In [ ]:
len(image)

In [ ]:
image[:2]

In [ ]:
# turn image into a tensor
tf.constant(image)[:2]

In [ ]:
# import tensorflow as tf

# Define Image size
IMG_SIZE = 528  # Adjust this as needed for your specific model

def process_image(image_path, img_size=IMG_SIZE):
    """
    Takes an Image File path and turns it into a Tensor.
    This function reads the image file, decodes it, resizes it to the desired size,
    and normalizes the pixel values to be in the range [0, 1].
    """
    try:
        # Read in an image file
        image = tf.io.read_file(image_path)
        # Turn the jpeg into a numerical tensor with 3 color channels (Red, Green, Blue)
        image = tf.image.decode_jpeg(image, channels=3)
        # Convert the color channel values from 0-255 to 0-1
        image = tf.image.convert_image_dtype(image, tf.float32)
        # Resize the image to our desired values (528, 528)
        image = tf.image.resize(image, size=[img_size, img_size])
        return image
    except tf.errors.InvalidArgumentError:
        print(f"Error: Unknown image file format for file {image_path}")
        return None

In [ ]:
# Create a simple function to return a tuple (image, label)
def get_image_label(image_path, label):
  image = process_image(image_path)
  return image, tf.squeeze(label)

In [ ]:
get_image_label(X[42], tf.constant(y[42]))

In [ ]:
BATCH_SIZE = 32

# Function to turn data into batches
def create_data_batches(X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
    """
    Create batches of data out of Images and Labels.
    Shuffle the data if it is Training but don't shuffle if it is Test or validation.
    Accepts test data as inputs.
    """
    # If we're dealing with test data we have no labels
    if test_data:
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
        data_batch = data.map(process_image).batch(BATCH_SIZE)
        return data_batch

    # If we're dealing with valid dataset we have labels and we don't need to shuffle it
    elif valid_data:
        print("Creating validation data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)  # Only batch once here
        return data_batch
    else:
        print("Creating training data batches...")
        # Turn filepath labels into tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))

        # Shuffling filepaths and labels before mapping `process_image` is faster than shuffling images
        data = data.shuffle(buffer_size=len(X))
        # Create tuples (image, label) and process the images from the path
        data = data.map(get_image_label).batch(BATCH_SIZE)  # Only batch once here
        return data

In [ ]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [ ]:
# Check the attributes of our data batches
train_data.element_spec, val_data.element_spec

In [ ]:
Image(X_train[64])

In [ ]:
type(train_data)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def show_25_image(images, labels, unique_labels):
    """
    Displays a plot of 25 images and their labels from a data batch
    """
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax = plt.subplot(5, 5, i+1)
        img = images[i, :, :, :]  # Extract the i-th image from the batch, removing the extra dimension
        plt.imshow(img)  # Display the image
        plt.title(unique_labels[labels[i].argmax()])
        plt.axis("off")
    plt.show()  # Display the plot

In [ ]:
train_image, train_label = next(train_data.as_numpy_iterator())
len(train_image), len(train_label)

In [ ]:
show_25_image(train_image, train_label, unique_labels)

In [ ]:
# Setup input shape to the model
INPUT_SHAPE = [IMG_SIZE, IMG_SIZE, 3]  # height, width, color channels

# Setup output shape of our model
OUTPUT_SHAPE = len(unique_labels)

# Setup model URL from TensorFlow Hub
# MODEL_URL = "https://www.kaggle.com/models/google/resnet-v1/TensorFlow2/101-classification/2"

In [ ]:
len(unique_labels)

In [ ]:
import tf_keras

In [ ]:
tf_keras.__version__

In [ ]:
import tensorflow_hub as hub
import tf_keras as keras  # Import tf_keras

# Kaggle-hosted EfficientNet URL
MODEL_URL = "https://www.kaggle.com/models/google/efficientnet/TensorFlow1/b6-classification/1"

def create_model(input_shape=(528, 528, 3), output_shape=3, model_url=MODEL_URL):
    print("Building model with:", model_url)

    # Setup the model layers for classification
    model = keras.Sequential([
        hub.KerasLayer(model_url, input_shape=input_shape, trainable=False),  # Set trainable to False
        keras.layers.Dense(units=output_shape, activation="softmax")
    ])

    # Compile the model
    model.compile(
        loss=keras.losses.CategoricalCrossentropy(),
        optimizer=keras.optimizers.Adam(),
        metrics=["accuracy"]
    )

    return model

# Define your input and output shapes
INPUT_SHAPE = (528, 528, 3)
OUTPUT_SHAPE = 3

model = create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL)
model.summary()

In [ ]:
print(train_data.element_spec)

In [ ]:
model = create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE, model_url=MODEL_URL)
model.summary()

In [ ]:
# Load TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
import datetime

# Create a function to build a TensorBoard callback
def create_tensorboard_callback():
  # Create a log directory for storing TensorBoard logs
  logdir = os.path.join("/kaggle/working/logs",
                        # Make it so the logs get tracked whenever we run an experiment
                        datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  return keras.callbacks.TensorBoard(logdir)

In [ ]:
# Create early stopping callback
early_stopping = keras.callbacks.EarlyStopping(monitor="val_f1_score",
                                                  patience=3)

In [ ]:
NUM_EPOCHS = 100

In [ ]:
# Check to make sure we're still running on a GPU
print("GPU", "available (YESSS!!!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")

In [ ]:
def train_model():
    # Create a model
    model = create_model()

    # Create new TensorBoard session everytime we train a model
    tensorboard = create_tensorboard_callback()

    # Reshape the target tensor to match the output shape
#     target_reshaped = tf.reshape(target, (-1, 4))

    # Fit the model to the data passing it the callbacks we created
    model.fit(train_data,
              epochs=NUM_EPOCHS,
              validation_data=val_data,
              validation_freq=1,
              callbacks=[tensorboard, early_stopping])
    # Return the fitted model
    return model

In [ ]:
train_data.element_spec

In [ ]:
print(train_data.cardinality())

In [ ]:
for batch in train_data.take(5):
    print(batch[0].shape, batch[0].dtype)
    print(batch[1].shape, batch[1].dtype)

In [ ]:
# Fit the model to the data
model = train_model()

In [ ]:
%tensorboard --logdir /kaggle/working/logs

In [ ]:
# Make predictions on the validation data (not used to train on)
predictions = model.predict(val_data, verbose=1) # verbose shows us how long there is to go
predictions

In [ ]:
# First prediction
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {unique_labels[np.argmax(predictions[0])]}") # the predicted label

In [ ]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_labels[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[0])
pred_label

In [ ]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
  """
  Takes a batched dataset of (image, label) Tensors and returns separate arrays
  of images and labels.
  """
  images = []
  labels = []
  # Loop through unbatched data
  for image, label in data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(unique_labels[np.argmax(label)])
  return images, labels

# Unbatchify the validation data
val_images, val_labels = unbatchify(val_data)
val_images[0], val_labels[0]

In [ ]:
def plot_pred(prediction_probabilities, labels, images, n=73):
  """
  View the prediction, ground truth label and image for sample n.
  """
  pred_prob, true_label, image = prediction_probabilities[n], labels[n], images[n]
  
  # Get the pred label
  pred_label = get_pred_label(pred_prob)
  
  # Plot image & remove ticks
  plt.imshow(image)
  plt.xticks([])
  plt.yticks([])

  # Change the color of the title depending on if the prediction is right or wrong
  if pred_label == true_label:
    color = "green"
  else:
    color = "red"

  plt.title("{} {:2.0f}% ({})".format(pred_label,
                                      np.max(pred_prob)*100,
                                      true_label),
                                      color=color)

In [ ]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          images=val_images)

In [ ]:
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
  modeldir = os.path.join("/kaggle/working/",
                          datetime.datetime.now().strftime("%Y%m%d-%H%M%s"))
  model_path = modeldir + "-" + suffix + ".h5" # save format of model
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
# Save our model
save_model(model, suffix="breast-Adam")

In [ ]:
%cd /kaggle/working

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink

FileLink(r'20240928-11161727522182-breast-Adam.h5')

In [ ]:
!zip -r file.zip /kaggle/working

In [ ]:
!ls

In [ ]:
from IPython.display import FileLink
FileLink(r'file.zip')

In [ ]:
import os

# Specify the file path
file_path = os.path.join('/kaggle/working/', '20240910-19521725997960-breast-Adam.h5')

# Check if the file exists
if os.path.exists(file_path):
    # Get the file size
    file_size = os.path.getsize(file_path)
    print(f"Size of {file_path}: {file_size / (1024 * 1024):.2f} MB")
else:
    print(f"File {file_path} does not exist.")